# Simulation command generation from simulation sheet
>'It's turtles all the way down!'

Plan: 
- Have array of sample settings, maybe optimized to give comparable $\sigma t$ to give comparable signal strength.

In [2]:
from definitions import s_t
import numpy as np


# Sample (R, t) [(m,m)]
samples = [(1e-6, 1e-3), (300e-9, 3e-3), (50e-9, 10e-3)]
# Source (L0, DL) [(m,m)]
sources = [(2.165, 0.02165), (4.321, 0.04321), (8, 0.8)]
# Universal sample parameters (?)
phi = 0.02

delta_rho = 2e14 # 1/m^2 (?)
for (R, t) in samples:
    print(F"R = {R * 1e9}nm, t = {t}m:")
    for (L0, DL) in sources:
        st = s_t(R,t, L0 * 1e-10, phi, delta_rho)
        permissible = st < 0.8 and st >= 0.1
        print(f"\t lambda = {L0} Å: sigma * t: {st}, permissible: {permissible}")
        sigma = s_t(R,t, L0 * 1e-10) / t
        t_optimum = 0.45 / sigma
        print(f"\t Optimal t: t={t_optimum}")
        # print(s_t(R,t_optimum, L0 * 1e-10))

R = 1000.0nm, t = 0.001m:
	 lambda = 2.165 Å: sigma * t: 0.055121766, permissible: False
	 Optimal t: t=0.0012033042843946863
	 lambda = 4.321 Å: sigma * t: 0.21957144215999996, permissible: True
	 Optimal t: t=0.0003020805280445737
	 lambda = 8 Å: sigma * t: 0.75264, permissible: True
	 Optimal t: t=8.812746756909194e-05
R = 300.0nm, t = 0.003m:
	 lambda = 2.165 Å: sigma * t: 0.0496095894, permissible: False
	 Optimal t: t=0.004011014281315621
	 lambda = 4.321 Å: sigma * t: 0.19761429794399998, permissible: True
	 Optimal t: t=0.0010069350934819121
	 lambda = 8 Å: sigma * t: 0.677376, permissible: True
	 Optimal t: t=0.0002937582252303065
R = 50.0nm, t = 0.01m:
	 lambda = 2.165 Å: sigma * t: 0.027560883, permissible: False
	 Optimal t: t=0.024066085687893726
	 lambda = 4.321 Å: sigma * t: 0.10978572107999997, permissible: True
	 Optimal t: t=0.006041610560891475
	 lambda = 8 Å: sigma * t: 0.37632, permissible: True
	 Optimal t: t=0.0017625493513818387


In [3]:
from definitions import s_t


# Sample (R, t) [(m,m)]
samples = [(1e-6, 1e-3), (300e-9, 1e-3), (50e-9, 10e-3)]
# Source (L0, DL) [(m,m)]
sources = [(4.321, 0.04321), (8, 0.8)]
# Universal sample parameters (?)
phi = 0.015

delta_rho = 2.08e14 # 1/m^2 (?)
for (R, t) in samples:
    print(F"R = {R * 1e9}nm, t = {t}m:")
    for (L0, DL) in sources:
        st = s_t(R,t, L0 * 1e-10, phi, delta_rho)
        permissible = st < 0.8 and st >= 0.1
        print(f"\t lambda = {L0} Å: ")
        print(f"\t\ts*t: {st}\n\t\t0.1 <= s*t <= 0.8: {permissible}")
        sigma = s_t(R,t, L0 * 1e-10) / t
        t_optimum = 0.45 / sigma
        # print(f"\t Optimal t: t={t_optimum}")
        # print(s_t(R,t_optimum, L0 * 1e-10))

R = 1000.0nm, t = 0.001m:
	 lambda = 4.321 Å: 
		s*t: 0.179025110787744
		0.1 <= s*t <= 0.8: True
	 lambda = 8 Å: 
		s*t: 0.613656576
		0.1 <= s*t <= 0.8: True
R = 300.0nm, t = 0.001m:
	 lambda = 4.321 Å: 
		s*t: 0.0537075332363232
		0.1 <= s*t <= 0.8: False
	 lambda = 8 Å: 
		s*t: 0.18409697279999998
		0.1 <= s*t <= 0.8: True
R = 50.0nm, t = 0.01m:
	 lambda = 4.321 Å: 
		s*t: 0.08951255539387198
		0.1 <= s*t <= 0.8: False
	 lambda = 8 Å: 
		s*t: 0.30682828799999995
		0.1 <= s*t <= 0.8: True


In [4]:
print("{:,.3e}".format(np.sqrt(6.1e-7) * 1e18))

7.810e+14


In [46]:
from definitions import s_t


# Sample (R, t) [(m,m)]
samples = [(1e-6, 1e-3), (300e-9, 1e-3), (50e-9, 10e-3)]
# Source (L0, DL) [(m,m)]
sources = [(4.321, 0.04321), (8, 0.8)]

# Source sample (thickness) pairing (L0, DL, R, t)
source_sample = [(4.321, 0.04321, 2e-6, 1e-3), (8, 0.8, 2e-6, 1e-3), (4.321, 0.04321, 300e-9, 10e-3), (8, 0.8, 300e-9, 5e-3), (4.321, 0.04321, 50e-9, 10e-3), (8, 0.8,50e-9, 10e-3)]
# Universal sample parameters (?)
phi = 0.015

delta_rho = 1.8e14 # 1/m^2 (?)
print(f"Constant parameters: drho = {delta_rho * 1e-14}e14 m^-2; phi = {phi}")
for (L0, DL, R, t) in source_sample:
    print(F"R = {R * 1e9}nm, t = {round(t * 1e3,2)}mm, lambda = {L0} Å:")
    st = s_t(R,t, L0 * 1e-10, phi, delta_rho)
    permissible = st < 0.8 and st >= 0.1
    print(f"\ts*t: {st}\n\t0.1 <= s*t <= 0.8: {permissible}")
    sigma = s_t(R,t, L0 * 1e-10) / t
    t_optimum = 0.38 / sigma
    # print(f"\tOptimal t: t={round(t_optimum * 1e3, 2)}mm")
    # print(s_t(R,t_optimum, L0 * 1e-10))

Constant parameters: drho = 1.9e14 m^-2; phi = 0.015
R = 2000.0nm, t = 1.0mm, lambda = 4.321 Å:
	s*t: 0.298761395129325
	0.1 <= s*t <= 0.8: True
R = 2000.0nm, t = 1.0mm, lambda = 8 Å:
	s*t: 1.0240848
	0.1 <= s*t <= 0.8: False
R = 300.0nm, t = 10.0mm, lambda = 4.321 Å:
	s*t: 0.4481420926939874
	0.1 <= s*t <= 0.8: True
R = 300.0nm, t = 5.0mm, lambda = 8 Å:
	s*t: 0.7680636
	0.1 <= s*t <= 0.8: True
R = 50.0nm, t = 10.0mm, lambda = 4.321 Å:
	s*t: 0.07469034878233123
	0.1 <= s*t <= 0.8: False
R = 50.0nm, t = 10.0mm, lambda = 8 Å:
	s*t: 0.2560212
	0.1 <= s*t <= 0.8: True


# Milk sample (from dairy paper)

In [36]:
delta_rho = 2.0e14 # 1/m^2 (?)
phi = 0.036
R = 50e-9
t = 2e-3
for (L0, _) in sources:
    st = s_t(R,t, L0 * 1e-10, phi, delta_rho)
    print(st)

0.03887758841183999
0.13326336
